In [2]:

def my_agent(obs, config):
    import numpy as np
    import random
    def visionDepth1(block_moves,obs,config):
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        bad_vision=[]
        obs_next=obs
        for i in block_moves:
            obs_next.board=np.asanyarray(drop_piece(grid, i, obs.mark, config)).reshape(config.rows*config.columns,1)
            op_moves = [col for col in range(config.columns) if obs_next.board[col] == 0]
            for j in op_moves:
                if check_winning_move(obs_next,config,j,oponent)==True:
                    bad_vision.append(i)
                    break
                    
        for k in bad_vision:
            block_moves.remove(k)
        return block_moves
    
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = piece
        return next_grid

    # Returns True if dropping piece in column results in game win
    def check_winning_move(obs, config, col, piece):
        # Convert the board to a 2D grid
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        next_grid = drop_piece(grid, col, piece, config)
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[row,col:col+config.inarow])
                if window.count(piece) == config.inarow:
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(next_grid[row:row+config.inarow,col])
                if window.count(piece) == config.inarow:
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        return False

    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
#     print(valid_moves)
    win_moves=[]
    for i in valid_moves:
        if check_winning_move(obs,config,i,obs.mark)==True:
            win_moves.append(i)
    # Your code here: Amend the agent!
#     print(win_moves)
    if win_moves !=[]:
        return random.choice(win_moves)
#     print(valid_moves)
    block_moves=[]
    for i in valid_moves:
        if obs.mark==1:
            oponent=2
        else: oponent=1
        if check_winning_move(obs,config,i,oponent)==True:
            block_moves.append(i)
    # Your code here: Amend the agent!
#     print(win_moves)
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    block_moves=visionDepth1(block_moves,obs,config)
    if block_moves !=[]:
        return random.choice(block_moves)
    least_depth=[]#try to go with col with vacancy
    depth_max=0
    
    for i in valid_moves:
#         colList=np.asarray([grid[_][i] for _ in range(config.rows)]).reshape(config.columns,1)
        colList=[grid[_][i] for _ in range(config.rows)]
        num_spaces=colList.count(0)
        if num_spaces>depth_max:
            depth_max=num_spaces
            least_depth=[i]
        elif num_spaces==depth_max:
            least_depth.append(i)
    least_depth=visionDepth1(least_depth,obs,config)
    if least_depth !=[]:
        return random.choice(least_depth)
    return random.choice(valid_moves)

ModuleNotFoundError: No module named 'kaggle_environments'

Review  
Current method only passively blocks loosing cases.  
## It does not :  
- acively search for winning case  
- prevent opponent's random winning case(lack of depth)  
## There are some winning hints:  
- two consecutive parts
- 33s  

## I can try:  
Removing shallow search - this was for preventing dead end(lacking of column choice)  
however random(which wasn't working properly in prev. ver) is now alive, and does kinda prevent former problem(lack of col choice).